In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
import warnings
import os


In [2]:
df1 = pd.read_excel('case_study1.xlsx')
df2 = pd.read_excel('case_study2.xlsx')

In [3]:
df1.sample(5)

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
11130,11131,2,2,0,0,0,0.000,0.000,0.000,1.000,...,0,0,2,0,0,2,0,0,63,63
8225,8226,1,1,0,0,0,0.000,0.000,0.000,1.000,...,0,1,0,0,0,0,1,0,31,31
47125,47126,7,4,3,0,0,0.000,0.000,0.429,0.571,...,0,0,1,0,0,7,0,2,140,8
25834,25835,5,1,4,4,1,0.800,0.200,0.800,0.200,...,0,1,0,0,2,2,3,1,10,3
38052,38053,7,4,3,6,4,0.857,0.571,0.429,0.571,...,0,1,0,0,2,4,3,4,6,3


In [4]:
df2.sample(5)

,PROSPECTID,time_since_recent_payment,time_since_first_deliquency,time_since_recent_deliquency,num_times_delinquent,max_delinquency_level,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,...,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
43764,43765,205,-99999,-99999,0,-99999,0,0,0,0,...,1.0,1.000,1.0,-99999.000,1,0,PL,ConsumerLoan,657,P4
25570,25571,50,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.000,0.0,2.001,0,0,others,others,710,P1
46325,46326,553,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.000,0.0,-99999.000,1,0,ConsumerLoan,others,683,P2
29713,29714,46,25,5,7,57,57,1,1,0,...,0.0,1.000,0.0,0.717,1,0,PL,AL,676,P2
783,784,77,-99999,-99999,0,-99999,0,0,0,0,...,0.5,0.375,0.5,0.389,0,0,others,PL,643,P4


In [5]:
df1 = df1.loc[df1['Age_Oldest_TL'] != -99999]
df1.shape

(51296, 26)

In [7]:
columns_to_remove = []
for i in df2.columns:
  if df2.loc[df2[i] == -99999].shape[0] > 10000:
    columns_to_remove.append(i)

In [8]:
columns_to_remove

['time_since_first_deliquency',
 'time_since_recent_deliquency',
 'max_delinquency_level',
 'max_deliq_6mts',
 'max_deliq_12mts',
 'CC_utilization',
 'PL_utilization',
 'max_unsec_exposure_inPct']

In [9]:
df2 = df2.drop(columns_to_remove,axis=1)

In [18]:
for i in df2.columns:
    df2 = df2.loc[ df2[i] != -99999 ]

In [11]:
for i in df1.columns:
  if i in df2.columns:
    print(i)

PROSPECTID


In [12]:
df = pd.merge(df1,df2,how='inner',left_on='PROSPECTID',right_on='PROSPECTID')

In [19]:
df.sample(10)

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
34313,41886,2,0,2,1,0,0.500,0.000,1.000,0.000,...,0.0,0.0,0.0,0.0,0,0,ConsumerLoan,ConsumerLoan,607,P4
23592,28823,2,2,0,0,0,0.000,0.000,0.000,1.000,...,0.0,0.0,0.0,0.0,1,0,ConsumerLoan,ConsumerLoan,676,P2
19113,23339,2,1,1,0,0,0.000,0.000,0.500,0.500,...,1.0,0.0,1.0,0.0,0,0,PL,PL,680,P2
17678,21582,9,6,3,1,1,0.111,0.111,0.333,0.667,...,0.0,0.0,0.0,0.0,1,0,others,ConsumerLoan,694,P2
40482,49404,2,0,2,2,0,1.000,0.000,1.000,0.000,...,1.0,0.0,1.0,0.0,0,0,ConsumerLoan,ConsumerLoan,631,P4
22261,27214,2,0,2,1,0,0.500,0.000,1.000,0.000,...,0.0,0.0,0.0,0.0,1,0,others,others,667,P3
19047,23250,6,4,2,1,0,0.167,0.000,0.333,0.667,...,1.0,0.0,1.0,0.0,1,0,ConsumerLoan,others,676,P2
3234,3898,1,0,1,0,0,0.000,0.000,1.000,0.000,...,0.0,0.0,0.0,0.0,0,0,ConsumerLoan,others,679,P2
40904,49916,1,0,1,0,0,0.000,0.000,1.000,0.000,...,0.0,0.0,0.0,0.0,0,0,others,others,677,P2
22116,27033,1,0,1,0,0,0.000,0.000,1.000,0.000,...,0.0,0.0,0.0,0.0,0,0,others,others,709,P1


In [20]:
df.shape

(42064, 79)

In [21]:
cat_col =[]
for i in df.columns:
  if df[i].dtypes == object:
    cat_col.append(i)
cat_col

['MARITALSTATUS',
 'EDUCATION',
 'GENDER',
 'last_prod_enq2',
 'first_prod_enq2',
 'Approved_Flag']

**Chi-square test**


In [22]:
for i in ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']:
    chi2, pval, u, l = chi2_contingency(pd.crosstab(df[i], df['Approved_Flag']))
    print(i, '---', pval)

MARITALSTATUS --- 3.578180861038862e-233
EDUCATION --- 2.6942265249737532e-30
GENDER --- 1.907936100186563e-05
last_prod_enq2 --- 0.0
first_prod_enq2 --- 7.84997610555419e-287


In [23]:
numeric_columns = []
for i in df.columns :
  if df[i].dtypes != object and i not in ['PROSPECTID','Approved_Flag']:
    numeric_columns.append(i)
len(numeric_columns)

72

**VIF sequentially check**


In [24]:
vif_data = df[numeric_columns]
total_columns = vif_data.shape[1]
columns_to_be_kept = []
column_index = 0


for i in range (0,total_columns):

    vif_value = variance_inflation_factor(vif_data, column_index)
    print (column_index,'---',vif_value)


    if vif_value <= 6:
        columns_to_be_kept.append( numeric_columns[i] )
        column_index = column_index+1

    else:
        vif_data = vif_data.drop([ numeric_columns[i] ] , axis=1)

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf
0 --- 11.320180023967996
0 --- 8.363698035000336
0 --- 6.520647877790928
0 --- 5.149501618212625
1 --- 2.611111040579735


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


2 --- inf
2 --- 1788.7926256209232
2 --- 8.601028256477228
2 --- 3.832800792153077
3 --- 6.099653381646723
3 --- 5.581352009642766
4 --- 1.985584353098778


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


5 --- inf
5 --- 4.80953830281934
6 --- 23.270628983464636
6 --- 30.595522588100053
6 --- 4.384346405965583
7 --- 3.0646584155234238
8 --- 2.898639771299251
9 --- 4.377876915347324
10 --- 2.207853583695844
11 --- 4.916914200506864
12 --- 5.214702030064725
13 --- 3.3861625024231476
14 --- 7.840583309478997
14 --- 5.255034641721434


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


15 --- inf
15 --- 7.380634506427238
15 --- 1.4210050015175733
16 --- 8.083255010190316
16 --- 1.6241227524040114
17 --- 7.257811920140003
17 --- 15.59624383268298
17 --- 1.825857047132431
18 --- 1.5080839450032664
19 --- 2.172088834824578
20 --- 2.6233975535272274
21 --- 2.2959970812106176
22 --- 7.360578319196446
22 --- 2.1602387773102567
23 --- 2.8686288267891467
24 --- 6.458218003637272
24 --- 2.8474118865638247
25 --- 4.753198156284083
26 --- 16.22735475594825
26 --- 6.424377256363877
26 --- 8.887080381808678
26 --- 2.3804746142952653
27 --- 8.60951347651454
27 --- 13.06755093547673
27 --- 3.500040056654653
28 --- 1.9087955874813773
29 --- 17.006562234161628
29 --- 10.730485153719197
29 --- 2.3538497522950275
30 --- 22.10485591513649
30 --- 2.7971639638512924
31 --- 3.424171203217696
32 --- 10.175021454450922
32 --- 6.408710354561292
32 --- 1.001151196262563
33 --- 3.069197305397273
34 --- 2.8091261600643724
35 --- 20.249538381980678
35 --- 15.864576541593774
35 --- 1.8331649740532

In [25]:
len(columns_to_be_kept)

39

**Check Anova for columns_to_be_kept**



In [26]:
from scipy.stats import f_oneway

columns_to_be_kept_numerical = []

for i in columns_to_be_kept:
    a = list(df[i])
    b = list(df['Approved_Flag'])

    group_P1 = [value for value, group in zip(a, b) if group == 'P1']
    group_P2 = [value for value, group in zip(a, b) if group == 'P2']
    group_P3 = [value for value, group in zip(a, b) if group == 'P3']
    group_P4 = [value for value, group in zip(a, b) if group == 'P4']


    f_statistic, p_value = f_oneway(group_P1, group_P2, group_P3, group_P4)

    if p_value <= 0.05:
        columns_to_be_kept_numerical.append(i)

In [27]:
len(columns_to_be_kept_numerical)

37

In [28]:
cat_col

['MARITALSTATUS',
 'EDUCATION',
 'GENDER',
 'last_prod_enq2',
 'first_prod_enq2',
 'Approved_Flag']

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 79 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PROSPECTID                  42064 non-null  int64  
 1   Total_TL                    42064 non-null  int64  
 2   Tot_Closed_TL               42064 non-null  int64  
 3   Tot_Active_TL               42064 non-null  int64  
 4   Total_TL_opened_L6M         42064 non-null  int64  
 5   Tot_TL_closed_L6M           42064 non-null  int64  
 6   pct_tl_open_L6M             42064 non-null  float64
 7   pct_tl_closed_L6M           42064 non-null  float64
 8   pct_active_tl               42064 non-null  float64
 9   pct_closed_tl               42064 non-null  float64
 10  Total_TL_opened_L12M        42064 non-null  int64  
 11  Tot_TL_closed_L12M          42064 non-null  int64  
 12  pct_tl_open_L12M            42064 non-null  float64
 13  pct_tl_closed_L12M          420

In [30]:
df.loc[df['EDUCATION'] == 'SSC',['EDUCATION']]              = 1
df.loc[df['EDUCATION'] == '12TH',['EDUCATION']]             = 2
df.loc[df['EDUCATION'] == 'GRADUATE',['EDUCATION']]         = 3
df.loc[df['EDUCATION'] == 'UNDER GRADUATE',['EDUCATION']]   = 3
df.loc[df['EDUCATION'] == 'POST-GRADUATE',['EDUCATION']]    = 4
df.loc[df['EDUCATION'] == 'OTHERS',['EDUCATION']]           = 1
df.loc[df['EDUCATION'] == 'PROFESSIONAL',['EDUCATION']]     = 3

df['EDUCATION'].value_counts()

EDUCATION
3    18931
2    11703
1     9532
4     1898
Name: count, dtype: int64

In [31]:
df = pd.get_dummies(df, columns=['MARITALSTATUS','GENDER', 'last_prod_enq2' ,'first_prod_enq2'])

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 91 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PROSPECTID                    42064 non-null  int64  
 1   Total_TL                      42064 non-null  int64  
 2   Tot_Closed_TL                 42064 non-null  int64  
 3   Tot_Active_TL                 42064 non-null  int64  
 4   Total_TL_opened_L6M           42064 non-null  int64  
 5   Tot_TL_closed_L6M             42064 non-null  int64  
 6   pct_tl_open_L6M               42064 non-null  float64
 7   pct_tl_closed_L6M             42064 non-null  float64
 8   pct_active_tl                 42064 non-null  float64
 9   pct_closed_tl                 42064 non-null  float64
 10  Total_TL_opened_L12M          42064 non-null  int64  
 11  Tot_TL_closed_L12M            42064 non-null  int64  
 12  pct_tl_open_L12M              42064 non-null  float64
 13  p

In [33]:
df['first_prod_enq2_others'] = df['first_prod_enq2_others'].astype(int)

In [34]:
df.first_prod_enq2_ConsumerLoan.value_counts()

first_prod_enq2_ConsumerLoan
False    30989
True     11075
Name: count, dtype: int64

**Creating new CSV file for for cleaned data**

In [35]:
df.to_csv('Clean_Bank_Data')

In [38]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['Approved_Flag'] = label_encoder.fit_transform(df['Approved_Flag'])

In [39]:
df.Approved_Flag.value_counts()

Approved_Flag
1    25452
2     6440
3     5264
0     4908
Name: count, dtype: int64

In [40]:
y = df['Approved_Flag']
x = df.drop(['Approved_Flag'],axis=1)

In [42]:
x.shape,y.shape

((42064, 90), (42064,))

# **Model Fitting**

In [44]:
y = df['Approved_Flag']
x = df.drop(['Approved_Flag'],axis=1)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test =train_test_split(x,y,test_size=0.2,random_state=786)

In [45]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=200,random_state=40)
model.fit(x_train,y_train)

RandomForestClassifier(n_estimators=200, random_state=40)

In [46]:
y_p = model.predict(x_test)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_p))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97      1046
           1       0.99      1.00      1.00      5073
           2       1.00      0.94      0.97      1284
           3       1.00      1.00      1.00      1010

    accuracy                           0.99      8413
   macro avg       0.98      0.98      0.98      8413
weighted avg       0.99      0.99      0.99      8413



In [62]:
from sklearn.tree import DecisionTreeClassifier
mode = DecisionTreeClassifier(criterion='gini',splitter='best', max_depth=25)
mode.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=25)

In [63]:
y_pred= mode.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1046
           1       1.00      1.00      1.00      5073
           2       1.00      1.00      1.00      1284
           3       1.00      1.00      1.00      1010

    accuracy                           1.00      8413
   macro avg       1.00      1.00      1.00      8413
weighted avg       1.00      1.00      1.00      8413



Decision  Tree Algorithm gave the best result on classification report .Therefore we can use it for further deployment process of the model